In [71]:
# things we need for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

import numpy as np
import random
from tensorflow import keras
from keras.models import load_model

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vivek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vivek\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [72]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [73]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

668 documents
49 classes ['CourseB.Tech', 'CourseB.TechInt', 'CourseM.Tech', 'CourseMBATech', 'CourseMCA', 'CoursePGDM', 'CoursePhD', 'accreditation', 'affirmative', 'btechandmbatechadmissions', 'generalfeestructure', 'goodbye', 'greeting', 'hours', 'integratedprograms', 'noanswer', 'offensivewords', 'opentoday', 'otherprogramsadmissions', 'postgradprograms', 'streamCivil', 'streamComputer', 'streamDataScience', 'streamElectrical', 'streamElectronicsandTelecommunications', 'streamIndustrialAutomation', 'streamInformationTechnology', 'streamMechanical', 'streamMechatronics', 'streamartificialintelligence', 'thanks', 'undergradprograms', 'whyArtificialIntelligence', 'whyB.Tech', 'whyB.TechIntegrated', 'whyCS', 'whyCivil', 'whyDataScience', 'whyEXTC', 'whyElectrical', 'whyIT', 'whyIndustrialAutomation', 'whyM.Tech', 'whyMBATech', 'whyMCA', 'whyMechanical', 'whyMechatronics', 'whyPGDM', 'whyPhD']
203 unique lemmatized words ['!', "'s", '.', 'a', 'about', 'acceptance', 'accreditation', 'adm

In [74]:
model = load_model('model_ChatBot.h5')

In [75]:
import re
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    
    ''''
    #converting streams to a unique form
    sentence_words = (str(sentence_words).lower())
    sentence_words = re.sub(r"i'm", "i am", sentence_words)
    sentence_words = re.sub(r"he's", "he is", sentence_words)
    sentence_words = re.sub(r"she's", "she is", sentence_words)
    sentence_words = re.sub(r"that's", "that is", sentence_words)
    sentence_words = re.sub(r"what's", "what is", sentence_words)
    sentence_words = re.sub(r"where's", "where is", sentence_words)
    sentence_words = re.sub(r"how's", "how is", sentence_words)
    sentence_words = re.sub(r"\'ll", " will", sentence_words)
    sentence_words = re.sub(r"\'ve", " have", sentence_words)
    sentence_words = re.sub(r"\'re", " are", sentence_words)
    sentence_words = re.sub(r"\'d", " would", sentence_words)
    sentence_words = re.sub(r"n't", " not", sentence_words)
    sentence_words = re.sub(r"won't", "will not", sentence_words)
    sentence_words = re.sub(r"can't", "cannot", sentence_words)
    sentence_words = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", sentence_words)
    sentence_words = re.sub(r"\b-tech", "btech", sentence_words)
    sentence_words = re.sub(r"\b.tech", "btech", sentence_words)
    sentence_words = re.sub(r"\mba-tech", "mbatech", sentence_words)
    sentence_words = re.sub(r"\mba.tech", "mbatech", sentence_words)
    sentence_words = re.sub(r"\b-tech integrated", "btech integrated", sentence_words)
    sentence_words = re.sub(r"\b.tech integrated", "btech integrated", sentence_words)
    sentence_words = re.sub(r"\b.tech int", "btech integrated", sentence_words)
    '''
    # lemmatize each word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [76]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.65
def classify(sentence):
    # generate probabilities from the model
    p = bow(sentence, words)
    
    d = len(p)
    f = len(documents)-2
    a = np.zeros([f, d])
    tot = np.vstack((p,a))
    
    results = model.predict(tot)[0]
    
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID, show_details=False):
    results = classify(sentence)
    print('Result:',results)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    #print(i)
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return (random.choice(i['responses']))
            results.pop(0)

In [77]:
response('hi', '123', show_details=True)

Result: [('greeting', 0.99999976)]
context: 
tag: greeting


'Hello, how can I help you?'

In [78]:
response('accreditations','123',show_details=True)

Result: [('accreditation', 0.99999976)]
tag: accreditation


'The Programs offered at MPSTME are All India Council for Technical Education(AICTE) Approved. Computer Engineering & Electronics and Telecommunication Programs have been accredited by National Board of Accreditation under Tier â€“ I for two years. (implies international accreditation)'

In [79]:
response('I want to know about the Undergraduate Programs in this college.','123',show_details=True)

Result: [('undergradprograms', 1.0)]
tag: undergradprograms


'We provide the following Undergraduate Programs - B. Tech (4 Years after 12th grade), B. Tech Integrated (6 Years after 10th grade)'

In [80]:
response('stream Mechanical','123',show_details=True )

Result: [('streamMechanical', 0.9999989)]
tag: streamMechanical


'To be a center of excellence in Mechanical Engineering for education and research to create competent Mechanical Engineering graduates who can be employable in the industries and be part of innovation, research, problem solving and entrepreneurship to help mankind in particular and society in general. We provide Mechanical Engineering Stream in the following courses: B.Tech, B. Tech Integrated, MBA Tech '

In [81]:
response('What is the college fees at MPSTME?','123',show_details=True)

Result: [('generalfeestructure', 0.9999995)]
tag: generalfeestructure


'Fee structure for various programs at MPSTME: MBA Tech - 3.3-3.7LPA, BTech - 2.7-3.0LPA, BTech Integrated - 2.7-3.0LPA, PHD - 0.5-0.7LPA, MTech - 1.2LPA, MCA - 1.1LPA, PGDM - 4.9LPA'

In [82]:
response('Tell me about B.Tech course','123',show_details=True)

Result: [('CourseB.Tech', 0.99999714)]
tag: CourseB.Tech


'Let me help you! We offer 4 year B. Tech Course for 12th+ Students at our Mumbai and Shirpur campuses.We provide B. Tech Degree in the following streams: Electrical (30 Seats), Data Science (60 Seats), - Information Technology (60+60 Seats), Computer Engineering (120+60 Seats, Civil (60 Seats), Electronics and Telecommunication (60+60 Seats), Mechatronics (60+60 Seats), Mechanical (60+60 Seats). *Seats: Mumbai + Shirpur; Minimum 70% in 12th grade required.'

In [83]:
response('b.tech admissions', '124',show_details=True )

Result: [('btechandmbatechadmissions', 0.9997774)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'

In [84]:
response('btech admissions', '124',show_details=True )

Result: [('btechandmbatechadmissions', 0.9999995)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'

In [85]:
response('B.Tech admissions', '124',show_details=True )

Result: [('btechandmbatechadmissions', 0.9997774)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'

In [86]:
response('b tech admissions', '124',show_details=True )

Result: [('btechandmbatechadmissions', 0.91803247)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'

In [87]:
response('B. Tech admissions', '124',show_details=True )

Result: [('btechandmbatechadmissions', 0.9665055)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'

In [88]:
response('B-tech admissions', '124',show_details=True )

Result: [('noanswer', 0.9784425)]
tag: noanswer


"Sorry, can't understand you"

In [89]:
response('B.Tech admissions', '124',show_details=True )

Result: [('btechandmbatechadmissions', 0.9997774)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'